If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

如果您在 colab 上打开此笔记本，您可能需要安装 🤗 Transformers 和 🤗 数据集。 取消注释以下单元格并运行它。

In [ ]:
! pip install transformers[torch]

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

如果您在本地打开此笔记本，请确保您的环境安装了这些库的最新版本。

为了能够与社区分享您的模型并通过推理 API 生成如下图所示的结果，还需要遵循几个步骤。

首先，您必须存储来自 Hugging Face 网站的身份验证令牌（如果尚未注册，请在[此处](https://huggingface.co/join) 注册！），然后执行以下单元格并输入您的用户名和密码：

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
# !apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

确保您的 Transformers 版本至少为 4.11.0，因为该版本中引入了该功能：

In [ ]:
import transformers

print(transformers.__version__)

You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

您可以找到此笔记本的脚本版本，以使用多个 GPU 或 TPU 以分布式方式微调您的模型 [此处](https://github.com/huggingface/transformers/tree/master/examples/language-modeling) 。


We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

我们还快速上传一些遥测数据 - 这告诉我们正在使用哪些示例和软件版本，以便我们知道在哪里优先考虑我们的维护工作。 我们不收集（或关心）任何个人身份信息，但如果您不想被统计在内，请随时跳过此步骤或完全删除此单元格。

In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_notebook", framework="pytorch")

# Fine-tuning a language model

In this notebook, we'll see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling tasks. We will cover two types of language modeling tasks which are:

在本笔记本中，我们将了解如何在语言建模任务中微调 [🤗 Transformers](https://github.com/huggingface/transformers) 模型之一。 我们将介绍两种类型的语言建模任务：

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

  因果语言建模：模型必须预测句子中的下一个标记（因此标签与向右移动的输入相同）。 为了确保模型不会作弊，它会获得一个注意掩码，以防止模型在尝试预测句子中的标记 i+1 时访问标记 i 之后的标记。

![Widget inference representing the causal language modeling task](images/causal_language_modeling.png)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.
  屏蔽语言建模：模型必须预测输入中屏蔽的一些标记。 它仍然可以访问整个句子，因此它可以使用被屏蔽的标记之前和之后的标记来预测它们的值。

![Widget inference representing the masked language modeling task](images/masked_language_modeling.png)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to fine-tune a model on it.

我们将了解如何轻松加载和预处理每一项任务的数据集，以及如何使用“Trainer” API 对其模型进行微调。

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

我们的 [examples 文件夹](https://github.com/huggingface/transformers/tree/master/examples) 中提供了可以直接在分布式环境或 TPU 上运行的此笔记本的脚本版本。

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

对于每一个任务，我们都将使用 Wikitext 数据集作为示例。 您可以使用 🤗 数据集库轻松加载它。

In [ ]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

您可以将上面的数据集替换为[the hub](https://huggingface.co/datasets)上托管的任何数据集或使用您自己的文件。 只需取消注释以下单元格并将路径替换为将指向您的文件的值：


In [ ]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

您还可以从 csv 或 JSON 文件加载数据集，有关更多信息，请参阅[完整文档](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files)。

To access an actual element, you need to select a split first, then give an index:

要访问实际元素，您需要先选择一个拆分，然后给出一个索引：

In [ ]:
datasets

In [ ]:
datasets["train"][10]

*To* get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

为了了解数据的样子，以下函数将显示数据集中随机选取的一些示例。

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

正如我们所看到的，一些文本是维基百科文章的完整段落，而另一些文本只是标题或空行。

## Causal Language modeling

因果语言建模

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:

对于因果语言建模（CLM），我们将获取数据集中的所有文本，并在标记化后将它们连接起来。 然后我们将它们分成一定序列长度的示例。 这样，模型将接收可能类似于以下内容的连续文本块：

```
part of text 1
```

or

```
end of text 1 [BOS_TOKEN] beginning of text 2
```

depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

取决于它们是否跨越数据集中的多个原始文本。 标签将与输入相同，但向左移动。

We will use the [`distilgpt2`](https://huggingface.co/distilgpt2) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead:

我们将在此示例中使用 [`distilgpt2`](https://huggingface.co/distilgpt2) 模型。 您可以选择[此处](https://huggingface.co/models?filter=causal-lm)列出的任何检查点：

In [ ]:
model_checkpoint = "distilgpt2"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

为了使用训练模型时使用的相同词汇来标记所有文本，我们必须下载预训练的标记器。 这一切都是由 AutoTokenizer 类完成的：

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

然后我们将其应用于“datasets”对象中的所有分割，使用“batched=True”和 4 个进程来加速预处理。 之后我们不需要“text”列，因此我们将其丢弃。

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

如果我们现在查看数据集的一个元素，我们将看到文本已被模型所需的“input_ids”替换：

In [ ]:
tokenized_datasets

In [ ]:
tokenized_datasets["train"][1]

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

现在是更困难的部分：我们需要将所有文本连接在一起，然后将结果分割成特定“block_size”的小块。 为此，我们将再次使用“map”方法，并使用选项“batched=True”。 该选项实际上允许我们通过返回与我们获得的示例数量不同的示例来更改数据集中的示例数量。 这样，我们就可以从一批示例中创建新示例。

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

首先，我们获取模型预训练的最大长度。 这可能太大了，无法放入您的 GPU RAM，所以这里我们只取 128。

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

首先请注意，我们复制了标签的输入。 这是因为🤗 Transformers 库的模型应用了向右移动，所以我们不需要手动执行。

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

另请注意，默认情况下，“map”方法将发送一批 1,000 个示例供预处理函数处理。 因此，在这里，我们将删除余数，以使连接的标记化文本每 1,000 个示例成为“block_size”的倍数。 您可以通过传递更高的批量大小来调整此行为（这也会导致处理速度变慢）。 您还可以使用多重处理来加速预处理：

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. We will a model:

现在数据已经清理完毕，我们准备实例化我们的“Trainer”。 我们将建立一个模型：

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

And some `TrainingArguments`:

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

最后一个参数用于设置所有内容，以便我们可以在训练期间定期将模型推送到 [Hub](https://huggingface.co/models)。 如果您没有按照笔记本顶部的安装步骤进行操作，请将其删除。 如果您想以与将要推送的存储库名称不同的名称在本地保存模型，或者如果您想将模型推送到组织而不是您的命名空间下，请使用 `hub_model_id` 参数来设置 存储库名称（它必须是全名，包括您的命名空间：例如“sgugger/gpt-finetuned-wikitext2”或“huggingface/gpt-finetuned-wikitext2”）。

We pass along all of those to the `Trainer` class:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

And we can train our model:

In [ ]:
trainer.train()

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

训练完成后，我们可以评估我们的模型并得到其在验证集上的困惑度，如下所示：

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

您现在可以与所有朋友、家人、最喜欢的宠物共享此模型：他们都可以使用标识符“您的用户名/您选择的名称”来加载它，例如：

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```

## Masked language modeling

掩码语言建模

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens).

对于屏蔽语言建模（MLM），我们将对数据集使用与之前相同的预处理，但需要执行一个额外步骤：我们将随机屏蔽一些标记（通过用“[MASK]”替换它们），并且标签将调整为仅 包括屏蔽标记（我们不必预测非屏蔽标记）。

We will use the [`distilroberta-base`](https://huggingface.co/distilroberta-base) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead:

我们将在此示例中使用 [`distilroberta-base`](https://huggingface.co/distilroberta-base) 模型。 您可以选择[此处](https://huggingface.co/models?filter=masked-lm)列出的任何检查点：

In [ ]:
model_checkpoint = "distilroberta-base"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

And like before, we group texts together and chunk them in samples of length `block_size`. You can skip that step if your dataset is composed of individual sentences.

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

We redefine our `TrainingArguments`:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

Like before, the last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-wikitext2"` or `"huggingface/bert-finetuned-wikitext2"`).

Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

最后，我们使用一个特殊的“data_collator”。 “data_collator”是一个函数，负责获取样本并以张量对它们进行批处理。 在前面的示例中，我们没有什么特别要做的，因此我们只使用此参数的默认值。 这里我们想要进行随机掩蔽。 我们可以将其作为预处理步骤（如标记化）来完成，但是标记在每个时期总是以相同的方式进行屏蔽。 通过在“data_collator”中执行此步骤，我们确保每次检查数据时都以新的方式完成随机屏蔽。

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking:

为了为我们进行这种屏蔽，该库提供了一个“DataCollatorForLanguageModeling”。 我们可以调整掩蔽的概率：

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to `Trainer` and begin training:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("sgugger/my-awesome-model")
```